In [1]:
import pandas as pd
import numpy as np

import pickle

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk
import spacy

First, pulling in modgroups. Contains ~296k unique names and mod classes. Will append this value to the final data

In [2]:
mod_groups = pd.read_csv('/Users/robertpagano/Desktop/AOC final graphs/Mod_class_marvel.csv')

In [3]:
with open('/Users/robertpagano/metis_data/project_5/final_raw_marvel_df.pickle', 'rb') as f:
    df_marvel = pickle.load(f)

In [4]:
df_marvel.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,...,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote
0,1107088444658978816,442928354,Erica Miller,EricaFMiller,Sun Mar 17 01:17:08 +0000 2019,RT @Raulc_52: Intro to Captain Marvel\nMe: htt...,0,5,128,False,...,13.0,407.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,1107088442356367360,408092583,Annie Rachel,Annferddd,Sun Mar 17 01:17:08 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,411,False,...,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,1107088441588830209,902039904099684352,𝙠𝙮𝙧𝙖 ♡,cptkyra,Sun Mar 17 01:17:07 +0000 2019,RT @lalisamcu: the gays when the scene cut to ...,0,26,147,False,...,97.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
3,1107088439382429696,985172108547309569,레돌샨,chisakaoo,Sun Mar 17 01:17:07 +0000 2019,RT @cyanwhisky: #AvengersEndgame #CaptainMarve...,0,13712,60,False,...,31450.0,26699.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,1107088438703128576,2864913304,dirty diana,dollyweep,Sun Mar 17 01:17:07 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,141,False,...,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [5]:
df_marvel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1779794 entries, 0 to 3167
Data columns (total 28 columns):
tweet_id                   int64
user_id                    int64
display_name               object
screen_name                object
tweet_date                 object
main_text                  object
fav_count                  int64
rt_count                   int64
followers_count            int64
is_verified                bool
reply_to_tweet_id          float64
reply_to_user_id           float64
reply_to_screen_name       object
retweet_text               object
retweet_id                 float64
retweet_user_id            float64
retweet_screen_name        object
retweet_is_verified        object
retweet_fav_count          float64
retweet_followers_count    float64
quoted_text                object
quoted_id                  float64
quoted_user_id             float64
quoted_screen_name         object
quoted_is_verified         object
quoted_fav_count           float64
is_r

In [6]:
df_marvel['screen_name'].nunique()

822604

In [7]:
mod_groups['Id'].nunique()

536602

Not sure why, but we must have cut out around 150k names from the network data. Should probably figure out who that is before continuing, I don't want to not include important accounts

In [8]:
mod_groups['modularity_class'].nunique()

16

In [9]:
mod_groups.groupby(['modularity_class']).count()[['Id']]

,Id
modularity_class,
0,152149
1,28090
2,129399
3,56875
4,103758
5,20012
6,28383
7,4123
8,23


SI can ignore:

- 8 
- 9
- 10
- 12
- 13
- 14
 
So we need to find the top accounts in each group, do NLP analysis of each group (topic modeling / sentiment analysis)


Next, to check - append community to each account. Then make sure the top x accounts have one of these communties

In [10]:
mod_groups.head()

,Id,Label,timeset,modularity_class,Weighted Degree
0,EricaFMiller,EricaFMiller,NaN,2,1
1,Raulc_52,Raulc_52,NaN,2,5
2,Annferddd,Annferddd,NaN,1,1
3,gwilymst,gwilymst,NaN,1,21603
4,cptkyra,cptkyra,NaN,2,1


In [11]:
mod_groups.drop(columns=['Id', 'timeset'], inplace=True)
mod_groups.head()

,Label,modularity_class,Weighted Degree
0,EricaFMiller,2,1
1,Raulc_52,2,5
2,Annferddd,1,1
3,gwilymst,1,21603
4,cptkyra,2,1


# TO DO

1. Append community to DF
2. Check top accounts are a part of community (and if they are, look back to see who we lost)
3. Topic modeling with LSA / NMF. Get top words each topic, etc.
4. New Jupyter Notebook - sentiment analysis on each topic by community

In [12]:
df_marvel = pd.merge(df_marvel, mod_groups, how='left', left_on='screen_name', right_on='Label')
df_marvel.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,...,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree
0,1107088444658978816,442928354,Erica Miller,EricaFMiller,Sun Mar 17 01:17:08 +0000 2019,RT @Raulc_52: Intro to Captain Marvel\nMe: htt...,0,5,128,False,...,NaN,NaN,NaN,NaN,NaN,1,0,EricaFMiller,2.0,1.0
1,1107088442356367360,408092583,Annie Rachel,Annferddd,Sun Mar 17 01:17:08 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,411,False,...,NaN,NaN,NaN,NaN,NaN,1,0,Annferddd,1.0,1.0
2,1107088441588830209,902039904099684352,𝙠𝙮𝙧𝙖 ♡,cptkyra,Sun Mar 17 01:17:07 +0000 2019,RT @lalisamcu: the gays when the scene cut to ...,0,26,147,False,...,NaN,NaN,NaN,NaN,NaN,1,0,cptkyra,2.0,1.0
3,1107088439382429696,985172108547309569,레돌샨,chisakaoo,Sun Mar 17 01:17:07 +0000 2019,RT @cyanwhisky: #AvengersEndgame #CaptainMarve...,0,13712,60,False,...,NaN,NaN,NaN,NaN,NaN,1,0,chisakaoo,0.0,1.0
4,1107088438703128576,2864913304,dirty diana,dollyweep,Sun Mar 17 01:17:07 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,141,False,...,NaN,NaN,NaN,NaN,NaN,1,0,dollyweep,1.0,1.0


In [13]:
pd.set_option('display.max_columns', 500)

In [14]:
df_marvel.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree
0,1107088444658978816,442928354,Erica Miller,EricaFMiller,Sun Mar 17 01:17:08 +0000 2019,RT @Raulc_52: Intro to Captain Marvel\nMe: htt...,0,5,128,False,NaN,NaN,None,Intro to Captain Marvel\nMe: https://t.co/WMJT...,1.107028e+18,2.344318e+09,Raulc_52,False,13.0,407.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,EricaFMiller,2.0,1.0
1,1107088442356367360,408092583,Annie Rachel,Annferddd,Sun Mar 17 01:17:08 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,411,False,NaN,NaN,None,captain marvel without context https://t.co/F...,1.106192e+18,1.042657e+18,gwilymst,False,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Annferddd,1.0,1.0
2,1107088441588830209,902039904099684352,𝙠𝙮𝙧𝙖 ♡,cptkyra,Sun Mar 17 01:17:07 +0000 2019,RT @lalisamcu: the gays when the scene cut to ...,0,26,147,False,NaN,NaN,None,the gays when the scene cut to brie larson rid...,1.105913e+18,9.735315e+17,lalisamcu,False,97.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,cptkyra,2.0,1.0
3,1107088439382429696,985172108547309569,레돌샨,chisakaoo,Sun Mar 17 01:17:07 +0000 2019,RT @cyanwhisky: #AvengersEndgame #CaptainMarve...,0,13712,60,False,NaN,NaN,None,#AvengersEndgame #CaptainMarvel OK I'm ready f...,1.106903e+18,1.966503e+08,cyanwhisky,False,31450.0,26699.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,chisakaoo,0.0,1.0
4,1107088438703128576,2864913304,dirty diana,dollyweep,Sun Mar 17 01:17:07 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,141,False,NaN,NaN,None,captain marvel without context https://t.co/F...,1.106192e+18,1.042657e+18,gwilymst,False,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,dollyweep,1.0,1.0


Now I want to put a counter for each screen name as a row

In [15]:
df_marvel['screenname_freq'] = df_marvel.groupby('screen_name')['screen_name'].transform('count')

In [16]:
df_marvel.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq
0,1107088444658978816,442928354,Erica Miller,EricaFMiller,Sun Mar 17 01:17:08 +0000 2019,RT @Raulc_52: Intro to Captain Marvel\nMe: htt...,0,5,128,False,NaN,NaN,None,Intro to Captain Marvel\nMe: https://t.co/WMJT...,1.107028e+18,2.344318e+09,Raulc_52,False,13.0,407.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,EricaFMiller,2.0,1.0,1
1,1107088442356367360,408092583,Annie Rachel,Annferddd,Sun Mar 17 01:17:08 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,411,False,NaN,NaN,None,captain marvel without context https://t.co/F...,1.106192e+18,1.042657e+18,gwilymst,False,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Annferddd,1.0,1.0,1
2,1107088441588830209,902039904099684352,𝙠𝙮𝙧𝙖 ♡,cptkyra,Sun Mar 17 01:17:07 +0000 2019,RT @lalisamcu: the gays when the scene cut to ...,0,26,147,False,NaN,NaN,None,the gays when the scene cut to brie larson rid...,1.105913e+18,9.735315e+17,lalisamcu,False,97.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,cptkyra,2.0,1.0,2
3,1107088439382429696,985172108547309569,레돌샨,chisakaoo,Sun Mar 17 01:17:07 +0000 2019,RT @cyanwhisky: #AvengersEndgame #CaptainMarve...,0,13712,60,False,NaN,NaN,None,#AvengersEndgame #CaptainMarvel OK I'm ready f...,1.106903e+18,1.966503e+08,cyanwhisky,False,31450.0,26699.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,chisakaoo,0.0,1.0,1
4,1107088438703128576,2864913304,dirty diana,dollyweep,Sun Mar 17 01:17:07 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,141,False,NaN,NaN,None,captain marvel without context https://t.co/F...,1.106192e+18,1.042657e+18,gwilymst,False,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,dollyweep,1.0,1.0,1


In [17]:
df_no_class = df_marvel[df_marvel['modularity_class'].isnull() == True]

In [18]:
df_no_class.sort_values('followers_count', ascending=False).head(20)

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq
289366,1106174135409958913,240649814,TIMES NOW,TimesNow,Thu Mar 14 12:44:00 +0000 2019,"The wait is finally over, and we have the new ...",29,4,8826177,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
1395127,1104246205683519488,16983244,MissMalini,MissMalini,Sat Mar 09 05:03:05 +0000 2019,I was super excited to see the movie @captainm...,29,1,2756909,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
670793,1105136344572014592,16807528,MTV UK,MTVUK,Mon Mar 11 16:00:11 +0000 2019,"Meet #GooseTheCat, the breakout star from #Cap...",15,4,2492731,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
509585,1105459287613231104,52833129,THE BEAT 99.9FM,THEBEAT999FM,Tue Mar 12 13:23:27 +0000 2019,Giveaway Time on the #MiddayShow!!! Two people...,35,2,1991522,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
759684,1104976836772225025,1430793200,ARY News,ARYNEWSOFFICIAL,Mon Mar 11 05:26:22 +0000 2019,Box Office: ‘#CaptainMarvel’ soars to $153 mil...,39,3,1748236,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
1658007,1103981187280728064,90347940,9XM,9xmHaqSe,Fri Mar 08 11:30:00 +0000 2019,Don't fear because #FilmyFriday is here. #Badl...,6,0,1741119,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
1376200,1104261784389611521,43855487,Business Standard,bsindia,Sat Mar 09 06:05:00 +0000 2019,"#CaptainMarvel is a thoroughly enjoyable film,...",11,0,1738514,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,2
1322476,1104343575381372928,43855487,Business Standard,bsindia,Sat Mar 09 11:30:00 +0000 2019,"#CaptainMavel takes a few turns, injects life ...",3,0,1738507,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,2
385925,1105812479886704640,16581734,HuffPost Parents,HuffPostParents,Wed Mar 13 12:46:54 +0000 2019,This photographer planned an awesome surprise ...,17,1,1621289,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
670739,1105136406186414082,25979455,Daily Nation,dailynation,Mon Mar 11 16:00:26 +0000 2019,RT @NationLifeStyle: 'Captain Marvel' soars to...,0,1,1602621,True,NaN,NaN,None,'Captain Marvel' soars to rarefied heights in ...,1.105136e+18,1.057959e+09,NationLifeStyle,True,4.0,19956.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,2


In [19]:
df_no_class.shape

(387076, 32)

In [20]:
df_marvel.shape

(1779794, 32)

In [21]:
509602-79489

430113

So it looks like the network communities just ignored those with very low frequency in the dataset, which is fine. If I ignore those without a modularity class, I end up with roughly 1.3 million observations (removing 307k). This should be fine for NLP analysis, so I'll just remove those

In [22]:
df_marvel = df_marvel[df_marvel['modularity_class'].isnull() == False]

In [23]:
df_marvel.shape
## And I'll pickle it

(1392718, 32)

In [24]:
with open('/Users/robertpagano/metis_data/project_5/marvel_nlp_null_class_removed.pickle', 'wb') as to_write:
    pickle.dump(df_marvel, to_write)

Now filter for just communties with enough data, and find the top posters in each

8, 9, 10, 12, 13, 14 can be ignored

In [25]:
# list of communities to run NLP on
list_of_coms = [0, 1, 2, 3, 4, 5, 6, 7, 11, 15]

In [26]:
df_marvel_coms = df_marvel[df_marvel['modularity_class'].isin(list_of_coms)]

In [27]:
df_marvel_coms.shape

(1390864, 32)

In [28]:
# get counts of tweets by modularity class
df_marvel_coms.groupby(['modularity_class']).count()[['screen_name']]

,screen_name
modularity_class,
0.0,454817
1.0,38344
2.0,396876
3.0,131119
4.0,247453
5.0,30183
6.0,69404
7.0,6789
11.0,1619


 - append retweet_mod_class to all retweets?
 - Append pagerank to each tweet, so I can get most influential nodes by class
 - sort by follower count first, then come back and add pagerank, betweenness, etc.

In [29]:
# So basically I need to do a groupby, screenname, follower count, head(10), and functionize it

In [30]:
df_marvel_coms.columns

Index(['tweet_id', 'user_id', 'display_name', 'screen_name', 'tweet_date',
       'main_text', 'fav_count', 'rt_count', 'followers_count', 'is_verified',
       'reply_to_tweet_id', 'reply_to_user_id', 'reply_to_screen_name',
       'retweet_text', 'retweet_id', 'retweet_user_id', 'retweet_screen_name',
       'retweet_is_verified', 'retweet_fav_count', 'retweet_followers_count',
       'quoted_text', 'quoted_id', 'quoted_user_id', 'quoted_screen_name',
       'quoted_is_verified', 'quoted_fav_count', 'is_retweet', 'is_quote',
       'Label', 'modularity_class', 'Weighted Degree', 'screenname_freq'],
      dtype='object')

I'm going to add pagerank here, and then add it to the "raw" file, and then re-run all the separate communities things

In [31]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/marvel_pagerank.pickle', 'rb') as f:
    pr = pickle.load(f)

In [32]:
df_marvel_coms['PageRank'] = df_marvel_coms['screen_name'].map(pr)
df_marvel_coms.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq,PageRank
0,1107088444658978816,442928354,Erica Miller,EricaFMiller,Sun Mar 17 01:17:08 +0000 2019,RT @Raulc_52: Intro to Captain Marvel\nMe: htt...,0,5,128,False,NaN,NaN,None,Intro to Captain Marvel\nMe: https://t.co/WMJT...,1.107028e+18,2.344318e+09,Raulc_52,False,13.0,407.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,EricaFMiller,2.0,1.0,1,9.162043e-07
1,1107088442356367360,408092583,Annie Rachel,Annferddd,Sun Mar 17 01:17:08 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,411,False,NaN,NaN,None,captain marvel without context https://t.co/F...,1.106192e+18,1.042657e+18,gwilymst,False,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Annferddd,1.0,1.0,1,6.333578e-07
2,1107088441588830209,902039904099684352,𝙠𝙮𝙧𝙖 ♡,cptkyra,Sun Mar 17 01:17:07 +0000 2019,RT @lalisamcu: the gays when the scene cut to ...,0,26,147,False,NaN,NaN,None,the gays when the scene cut to brie larson rid...,1.105913e+18,9.735315e+17,lalisamcu,False,97.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,cptkyra,2.0,1.0,2,5.114840e-07
3,1107088439382429696,985172108547309569,레돌샨,chisakaoo,Sun Mar 17 01:17:07 +0000 2019,RT @cyanwhisky: #AvengersEndgame #CaptainMarve...,0,13712,60,False,NaN,NaN,None,#AvengersEndgame #CaptainMarvel OK I'm ready f...,1.106903e+18,1.966503e+08,cyanwhisky,False,31450.0,26699.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,chisakaoo,0.0,1.0,1,5.573665e-07
4,1107088438703128576,2864913304,dirty diana,dollyweep,Sun Mar 17 01:17:07 +0000 2019,RT @gwilymst: captain marvel without context ...,0,32858,141,False,NaN,NaN,None,captain marvel without context https://t.co/F...,1.106192e+18,1.042657e+18,gwilymst,False,70766.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,dollyweep,1.0,1.0,1,6.333578e-07


In [33]:
df_coms_raw = df_marvel_coms.loc[:, ['screen_name', 'followers_count', 'modularity_class', 'main_text', 'retweet_text', 'tweet_date', 'PageRank', 'rt_count']]

In [34]:
df_com_0 = df_coms_raw[df_coms_raw['modularity_class'] == 0]
df_com_1 = df_coms_raw[df_coms_raw['modularity_class'] == 1]
df_com_2 = df_coms_raw[df_coms_raw['modularity_class'] == 2]
df_com_3 = df_coms_raw[df_coms_raw['modularity_class'] == 3]
df_com_4 = df_coms_raw[df_coms_raw['modularity_class'] == 4]
df_com_5 = df_coms_raw[df_coms_raw['modularity_class'] == 5]
df_com_6 = df_coms_raw[df_coms_raw['modularity_class'] == 6]
df_com_7 = df_coms_raw[df_coms_raw['modularity_class'] == 7]
df_com_11 = df_coms_raw[df_coms_raw['modularity_class'] == 11]
df_com_15 = df_coms_raw[df_coms_raw['modularity_class'] == 15]


In [35]:
print(df_com_0.shape)
print(df_com_1.shape)
print(df_com_2.shape)
print(df_com_3.shape)
print(df_com_4.shape)
print(df_com_5.shape)
print(df_com_6.shape)
print(df_com_7.shape)
print(df_com_11.shape)
print(df_com_15.shape)


(454817, 8)
(38344, 8)
(396876, 8)
(131119, 8)
(247453, 8)
(30183, 8)
(69404, 8)
(6789, 8)
(1619, 8)
(14260, 8)


In [36]:
df_com_0 = df_com_0.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_1 = df_com_1.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_2 = df_com_2.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_3 = df_com_3.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_4 = df_com_4.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_5 = df_com_5.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_6 = df_com_6.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_7 = df_com_7.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_11 = df_com_11.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_15 = df_com_15.sort_values(by=['PageRank', 'rt_count'], ascending=False)


In [37]:
df_com_0.head()

,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count
235359,brielarson,1243167,0.0,This! THIS!!!! https://t.co/eFymcwxozQ,NaN,Thu Mar 14 19:08:08 +0000 2019,0.009626,35792
1089633,brielarson,1244471,0.0,"RT @captainmarvel: #CaptainMarvel herself, @br...","#CaptainMarvel herself, @brielarson, popped in...",Sun Mar 10 04:35:44 +0000 2019,0.009626,21037
1221001,brielarson,1244700,0.0,RT @heyjenbartel: Happy International Women’s ...,Happy International Women’s Day! 💃🌟 https://t....,Sat Mar 09 19:39:53 +0000 2019,0.009626,18563
1521772,brielarson,1244972,0.0,Happy International Women’s Day to every type ...,NaN,Fri Mar 08 21:15:05 +0000 2019,0.009626,13558
1089825,brielarson,1244471,0.0,RT @captainmarvel: The ultimate #CaptainMarvel...,The ultimate #CaptainMarvel experience! @BrieL...,Sun Mar 10 04:35:18 +0000 2019,0.009626,11198


For the below - I want to create this with just a list of the most influential 20 in each category, for EDA purposes

In [38]:
df_com_0_top_20 = df_com_0.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_1_top_20 = df_com_1.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_2_top_20 = df_com_2.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_3_top_20 = df_com_3.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_4_top_20 = df_com_4.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_5_top_20 = df_com_5.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_6_top_20 = df_com_6.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_7_top_20 = df_com_7.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_11_top_20 = df_com_11.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_15_top_20 = df_com_15.drop_duplicates(subset='screen_name', keep='first').head(20)


In [39]:
df_com_5_top_20

,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count
1190624,9GAG,14991855,5.0,Do you know how i got these scars?\n#captainma...,NaN,Sat Mar 09 22:02:01 +0000 2019,0.006216,4723
418371,iamgeekingout,34696,5.0,RT @mcucevans: brie larson just ended the war ...,brie larson just ended the war between marvel ...,Wed Mar 13 02:53:35 +0000 2019,0.001789,5509
720317,Drian_Bautista,98183,5.0,Captain Mavel 💙❤️💛✨\n\nInspired by Captain mar...,NaN,Mon Mar 11 11:22:47 +0000 2019,0.001388,1924
1322607,mzieeezul,914,5.0,hall 2\nmovie: captain marvel\ntime: 4.20p.m\n...,NaN,Sat Mar 09 11:29:08 +0000 2019,0.001053,2125
978057,NajibRazak,4090042,5.0,"Kiky, are you a cat or Flerken? \n\n#CaptainMa...",NaN,Sun Mar 10 15:02:00 +0000 2019,0.000868,1942
102639,AmericanQueen18,3644,5.0,RT @gwilymst: captain marvel without context ...,captain marvel without context https://t.co/F...,Sat Mar 16 02:06:04 +0000 2019,0.000846,33070
1652861,TwtMCU,1337,5.0,Senarai movie Marvel mengikut kronologi oleh K...,NaN,Fri Mar 08 12:06:59 +0000 2019,0.000758,1767
753299,marvelstudiosph,40447,5.0,Just one of many feats on the road to becoming...,NaN,Mon Mar 11 06:10:40 +0000 2019,0.000316,622
1060872,thisisinsider,323710,5.0,"""We heard another moviegoer say Brie Larson is...",NaN,Sun Mar 10 06:15:01 +0000 2019,0.000260,151
1661478,ijaicool,147221,5.0,Thanos after watching Captain Marvel: https://...,NaN,Fri Mar 08 11:02:41 +0000 2019,0.000221,648


Pickling all of my community partitioned data, will begin work on topic modeling in new notebook

In [40]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_0.pickle', 'wb') as to_write:
    pickle.dump(df_com_0, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_1.pickle', 'wb') as to_write:
    pickle.dump(df_com_1, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_2.pickle', 'wb') as to_write:
    pickle.dump(df_com_2, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_3.pickle', 'wb') as to_write:
    pickle.dump(df_com_3, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_4.pickle', 'wb') as to_write:
    pickle.dump(df_com_4, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_5.pickle', 'wb') as to_write:
    pickle.dump(df_com_5, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_6.pickle', 'wb') as to_write:
    pickle.dump(df_com_6, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_7.pickle', 'wb') as to_write:
    pickle.dump(df_com_7, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_11.pickle', 'wb') as to_write:
    pickle.dump(df_com_11, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/marvel/community_pickles_pre_nlp/df_15.pickle', 'wb') as to_write:
    pickle.dump(df_com_15, to_write)
    